In [1]:
# Installar pip isntall psycopg2-binary
import psycopg2
import pandas as pd

# Insertar csv a postgresql


In [2]:
data = pd.read_csv('./scrap_data.csv', encoding='latin-1')

data.head()


,production_family,wk,fecha,fa_pn,pn_componente,wo,quantity,cod_defecto,causa,linea,turno
0,Wands (Kamuk),Week 01,1/3/2020,48400,48404,2046690.0,1,D1093,C7-Proceso,Line 13,T1
1,Wands (Kamuk),Week 01,1/3/2020,48400,48414,2046690.0,1,DO085,C7-Proceso,Line 13,T 1
2,Wands (Kamuk),Week 01,1/3/2020,48400,48416,2046690.0,1,DO1646,C7-Proceso,Line 13,T 1
3,Wands (Kamuk),Week 01,1/3/2020,48400,49697,2046690.0,4,DO1644,C7-Proceso,Line 13,T 1
4,Wands (Kamuk),Week 01,1/3/2020,48400,20363,2046690.0,4,NaN,C17- Pruebas de calidad,Line 13,T 1


# Correr desde aqui

In [3]:
create_query = 'DROP TABLE IF EXISTS bigdata.scrap;\
         CREATE TABLE bigdata.scrap\
        (\
        production_family varchar(256),wk varchar(256), fecha varchar(256), fa_pn varchar(256), pn_componente varchar(256), wo varchar(256),\
       quantity float, cod_defecto varchar(256), causa varchar(256), linea varchar(256), turno varchar(256)\
    ); \
commit;' 
create_query

'DROP TABLE IF EXISTS bigdata.scrap;         CREATE TABLE bigdata.scrap        (        production_family varchar(256),wk varchar(256), fecha varchar(256), fa_pn varchar(256), pn_componente varchar(256), wo varchar(256),       quantity float, cod_defecto varchar(256), causa varchar(256), linea varchar(256), turno varchar(256)    ); commit;'

In [6]:
con = psycopg2.connect( host='localhost', user='postgres', password='root', dbname='postgres', port = 5432 )
cursor=con.cursor()

In [7]:
cursor.execute(create_query)

In [8]:
pd.read_sql('select * from bigdata.scrap', con)


,production_family,wk,fecha,fa_pn,pn_componente,wo,quantity,cod_defecto,causa,linea,turno


In [10]:
with open('scrap_data.csv', 'r') as f:
# Notice that we don't need the `csv` module.
    next(f) # Skip the header row.
    cursor.copy_from(f, 'bigdata.scrap', sep=',')
    con.commit()


In [11]:
pd.read_sql('select * from bigdata.scrap', con)

,production_family,wk,fecha,fa_pn,pn_componente,wo,quantity,cod_defecto,causa,linea,turno
0,Wands (Kamuk),Week 01,1/3/2020,48400,48404,2046690.0,1.0,D1093,C7-Proceso,Line 13,T1
1,Wands (Kamuk),Week 01,1/3/2020,48400,48414,2046690.0,1.0,DO085,C7-Proceso,Line 13,T 1
2,Wands (Kamuk),Week 01,1/3/2020,48400,48416,2046690.0,1.0,DO1646,C7-Proceso,Line 13,T 1
3,Wands (Kamuk),Week 01,1/3/2020,48400,49697,2046690.0,4.0,DO1644,C7-Proceso,Line 13,T 1
4,Wands (Kamuk),Week 01,1/3/2020,48400,20363,2046690.0,4.0,,C17- Pruebas de calidad,Line 13,T 1
...,...,...,...,...,...,...,...,...,...,...,...
39849,SA,Week 04,1/26/2020,16358,7229,2049249.0,3.0,DW07,C7-Proceso,Returns,T 1
39850,SA,Week 04,1/26/2020,93846,42003,2048805.0,8.0,DW01,C7-Proceso,SA,T 1
39851,SA,Week 04,1/26/2020,93846,4496,2048805.0,6.0,,C11 - Material afectado por otro componente (n...,SA,T 1
39852,SA,Week 04,1/26/2020,93846,12533,2048805.0,6.0,,C11 - Material afectado por otro componente (n...,SA,T 1


In [12]:
con.close()

# Aqui ya se conecta con pyspark

In [13]:
import findspark
findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "./postgresql-42.2.8.jar") \
    .config("spark.executor.extraClassPath", "./postgresql-42.2.8.jar") \
    .getOrCreate()

# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/") \
    .option("user", "postgres") \
    .option("password", "root") \
    .option("dbtable", "bigdata.scrap")\
    .load()

In [14]:
df.show()

+-----------------+-------+--------+-----+-------------+---------+--------+-----------+--------------------+-------+-----+
|production_family|     wk|   fecha|fa_pn|pn_componente|       wo|quantity|cod_defecto|               causa|  linea|turno|
+-----------------+-------+--------+-----+-------------+---------+--------+-----------+--------------------+-------+-----+
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        48404|2046690.0|     1.0|      D1093|          C7-Proceso|Line 13|   T1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        48414|2046690.0|     1.0|      DO085|          C7-Proceso|Line 13|  T 1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        48416|2046690.0|     1.0|     DO1646|          C7-Proceso|Line 13|  T 1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        49697|2046690.0|     4.0|     DO1644|          C7-Proceso|Line 13|  T 1|
|    Wands (Kamuk)|Week 01|1/3/2020|48400|        20363|2046690.0|     4.0|           |C17- Pruebas de c...|Line 13|  T 1|
|    Wands (Kamu